In [12]:
%pip install sparkmagic
%pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [36]:
import pandas
import boto3
import io
client = boto3.client('kinesis',region_name="eu-west-3")

s3 = boto3.resource('s3')
s3_object = s3.Object('nvabucket', 'flights.csv')

In [37]:
csv_data = s3_object.get()['Body'].read().decode('utf-8')

In [53]:
df = pd.read_csv(io.StringIO(csv_data))

In [52]:
df

,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay
0,19,5,DL,11433,13303,-3,1
1,19,5,DL,14869,12478,0,-8
2,19,5,DL,14057,14869,-4,-15
3,19,5,DL,15016,11433,28,24
4,19,5,DL,11193,12892,-6,-11
...,...,...,...,...,...,...,...
2702213,19,6,DL,13204,14869,1,-3
2702214,19,6,DL,10397,13495,-2,-3
2702215,19,6,DL,10140,10397,18,9
2702216,19,6,DL,10397,13244,10,7


In [56]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sqlfunc
from pyspark.sql.types import *
import argparse, sys
from pyspark.sql import *
import pyspark.sql.functions as sqlfunc
import pandas as pd

def create_session(appname):
    spark_session = SparkSession\
        .builder\
        .appName(appname)\
        .master('yarn')\
        .config("hive.metastore.uris", "thrift://uds-far-mn1.dab.02.net:9083")\
        .enableHiveSupport()\
        .getOrCreate()
    return spark_session
### START MAIN ###
if __name__ == '__main__':
    spark_session = create_session('testing_files')

In [57]:
from pyspark.sql import SparkSession
# Pandas to Spark
csv = spark_session.createDataFrame(df)

In [58]:
csv.show(10)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

In [59]:
# Select the columns we want to use
# data = csv.select("DayofMonth", "DayOfWeek", "Carrier", "OriginAirportID", "DestAirportID", "DepDelay", ((col("ArrDelay") > 15).cast("Int").alias("label")))
data = (csv
        .withColumn("DayofMonth", col("DayofMonth").cast(DoubleType()))
        .withColumn("DayOfWeek", col("DayOfWeek").cast(DoubleType()))
        .withColumn("OriginAirportID", col("OriginAirportID").cast(IntegerType()))
        .withColumn("DestAirportID", col("DestAirportID").cast(IntegerType()))
        .withColumn("DepDelay", col("DepDelay").cast("float"))
        .select("DayofMonth", "DayOfWeek", "Carrier", "OriginAirportID", "DestAirportID", "DepDelay", ((col("ArrDelay") > 15).cast("Int").alias("label"))))

print("Select the columns we want to use")

Select the columns we want to use


In [61]:
# Split the data into train and test
[train, test] = data.randomSplit([0.7, 0.3])
test = test.withColumnRenamed("label", "trueLabel")

print("Split the data into train and test")

Split the data into train and test


In [60]:
# Create the pipeline
strIdx = StringIndexer(inputCol = "Carrier", outputCol = "CarrierIdx")
catVec = VectorAssembler(inputCols = ["CarrierIdx", "DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID"], outputCol="catFeatures")
catIdx = VectorIndexer(inputCol = catVec.getOutputCol(), outputCol = "idxCatFeatures")
numVec = VectorAssembler(inputCols = ["DepDelay"], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVec.getOutputCol(), outputCol="normFeatures")
featVec = VectorAssembler(inputCols=["idxCatFeatures", "normFeatures"], outputCol="features")
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)

In [62]:
#dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")
pipeline = Pipeline(stages=[strIdx, catVec, catIdx, numVec, minMax, featVec, lr])

print("Create the pipeline")

Create the pipeline


In [63]:
# Train the model
pipelineModel = pipeline.fit(train)

print("Train the model")

Train the model


In [64]:
# Make predictions
prediction = pipelineModel.transform(test)

print("Make predictions")

Make predictions


In [65]:
# Evaluate the model
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
aur = evaluator.evaluate(prediction)
print ("AUR = ", aur)

AUR =  0.9225174722680995


In [66]:
# Tune the model
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.3, 0.1]).addGrid(lr.maxIter, [10, 5]).addGrid(lr.threshold, 
                                                                                            [0.4, 0.3]).build()
cv = CrossValidator(estimator=pipeline, evaluator=BinaryClassificationEvaluator(), estimatorParamMaps=paramGrid, 
                    numFolds=2)

model = cv.fit(train)

In [67]:
# Make predictions
newPrediction = model.transform(test)

In [68]:
# Evaluate the model
evaluator2 = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="prediction", metricName="areaUnderROC")
aur2 = evaluator2.evaluate(newPrediction)
print( "AUR2 = ", aur2)

AUR2 =  0.6888168716879202


In [84]:
type(model)

pyspark.ml.tuning.CrossValidatorModel

In [81]:
s3_object = s3.Object('nvabucket', 'flights.csv')
model.save(s3_object)

print("Save the model")

TypeError: path should be a string, got type <class 'boto3.resources.factory.s3.Object'>